Anova_tfidf_100features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/tfidf_100_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,-0.08608,-0.08608,-0.197016,-0.08608,-0.093554,-0.113621,-0.331375,-0.131673,-0.11957,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608
1,1,-0.08608,-0.08608,4.406590,-0.08608,-0.093554,-0.113621,-0.331375,-0.131673,-0.11957,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608
2,2,-0.08608,-0.08608,-0.197016,-0.08608,-0.093554,-0.113621,-0.331375,-0.131673,-0.11957,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608
3,3,-0.08608,-0.08608,-0.197016,-0.08608,-0.093554,-0.113621,2.967225,-0.131673,-0.11957,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608
4,4,-0.08608,-0.08608,-0.197016,-0.08608,-0.093554,-0.113621,-0.331375,-0.131673,-0.11957,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head() 

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.08608,-0.08608,-0.197016,-0.08608,-0.093554,-0.113621,-0.331375,-0.131673,-0.11957,-0.08608,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608
1,-0.08608,-0.08608,4.406590,-0.08608,-0.093554,-0.113621,-0.331375,-0.131673,-0.11957,-0.08608,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608
2,-0.08608,-0.08608,-0.197016,-0.08608,-0.093554,-0.113621,-0.331375,-0.131673,-0.11957,-0.08608,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608
3,-0.08608,-0.08608,-0.197016,-0.08608,-0.093554,-0.113621,2.967225,-0.131673,-0.11957,-0.08608,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608
4,-0.08608,-0.08608,-0.197016,-0.08608,-0.093554,-0.113621,-0.331375,-0.131673,-0.11957,-0.08608,...,-0.104749,-0.08608,-0.08608,-0.08149,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608,-0.08608


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 100), (524, 100))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [15]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [16]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.35067006718964366


**Decision Tree**

In [19]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [20]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [21]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [22]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [23]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [24]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-23-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.484426,0.484426,0.584693,0.366152,0.425573,0.425573,0.283621,0.291643


**Random Forest**

In [25]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [26]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [27]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [28]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9882    0.6953    0.8163       361
           2     0.9974    0.6817    0.8099       553
           3     0.5204    1.0000    0.6846       306

    accuracy                         0.7656      1220
   macro avg     0.8353    0.7923    0.7702      1220
weighted avg     0.8750    0.7656    0.7803      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2605    0.1975    0.2246       157
           2     0.4855    0.3544    0.4098       237
           3     0.2759    0.4923    0.3536       130

    accuracy                         0.3416       524
   macro avg     0.3406    0.3481    0.3293       524
weighted avg     0.3661    0.3416    0.3404       524



In [29]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-23-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.484426,0.484426,0.584693,0.366152,0.425573,0.425573,0.283621,0.291643
1,RandomForest,0.765574,0.765574,0.875015,0.780337,0.341603,0.341603,0.366100,0.340357


**AdaBoost**

In [30]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5238    0.2133    0.3031       361
           2     0.5000    0.8807    0.6379       553
           3     0.5556    0.1797    0.2716       306

    accuracy                         0.5074      1220
   macro avg     0.5265    0.4246    0.4042      1220
weighted avg     0.5210    0.5074    0.4470      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.1867    0.0892    0.1207       157
           2     0.4458    0.7806    0.5675       237
           3     0.2647    0.0692    0.1098       130

    accuracy                         0.3969       524
   macro avg     0.2991    0.3130    0.2660       524
weighted avg     0.3232    0.3969    0.3201       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.481967,0.481967,0.633070,0.348368,0.433206,0.433206,0.279998,0.284854
1,RandomForest,0.821311,0.821311,0.868298,0.818609,0.400763,0.400763,0.368614,0.373549
2,Adaboost,0.507377,0.507377,0.520980,0.446951,0.396947,0.396947,0.323223,0.320058


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.8074    0.3019    0.4395       361
           2     0.5406    0.9873    0.6987       553
           3     0.9600    0.2353    0.3780       306

    accuracy                         0.5959      1220
   macro avg     0.7693    0.5082    0.5054      1220
weighted avg     0.7247    0.5959    0.5415      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2179    0.1083    0.1447       157
           2     0.4523    0.7806    0.5728       237
           3     0.1892    0.0538    0.0838       130

    accuracy                         0.3989       524
   macro avg     0.2865    0.3142    0.2671       524
weighted avg     0.3168    0.3989    0.3232       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.481967,0.481967,0.633070,0.348368,0.433206,0.433206,0.279998,0.284854
1,RandomForest,0.821311,0.821311,0.868298,0.818609,0.400763,0.400763,0.368614,0.373549
2,Adaboost,0.507377,0.507377,0.520980,0.446951,0.396947,0.396947,0.323223,0.320058
3,GBM,0.595902,0.595902,0.724740,0.541538,0.398855,0.398855,0.316819,0.323199


**SVM Model**

In [31]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [32]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [33]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [34]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3789    0.2382    0.2925       361
           2     0.5526    0.3797    0.4502       553
           3     0.3067    0.6144    0.4091       306

    accuracy                         0.3967      1220
   macro avg     0.4127    0.4108    0.3839      1220
weighted avg     0.4395    0.3967    0.3932      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2521    0.1911    0.2174       157
           2     0.4797    0.2996    0.3688       237
           3     0.2724    0.5385    0.3618       130

    accuracy                         0.3263       524
   macro avg     0.3347    0.3430    0.3160       524
weighted avg     0.3601    0.3263    0.3217       524



In [35]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-23-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.484426,0.484426,0.584693,0.366152,0.425573,0.425573,0.283621,0.291643
1,RandomForest,0.765574,0.765574,0.875015,0.780337,0.341603,0.341603,0.366100,0.340357
2,SVM,0.396721,0.396721,0.439523,0.393225,0.326336,0.326336,0.360085,0.321702


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4525    0.3296    0.3814       361
           2     0.5083    0.8282    0.6300       553
           3     0.5357    0.0980    0.1657       306

    accuracy                         0.4975      1220
   macro avg     0.4988    0.4186    0.3924      1220
weighted avg     0.4987    0.4975    0.4400      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2783    0.2038    0.2353       157
           2     0.4514    0.7257    0.5566       237
           3     0.3214    0.0692    0.1139       130

    accuracy                         0.4065       524
   macro avg     0.3504    0.3329    0.3020       524
weighted avg     0.3673    0.4065    0.3505       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.481967,0.481967,0.633070,0.348368,0.433206,0.433206,0.279998,0.284854
1,RandomForest,0.821311,0.821311,0.868298,0.818609,0.400763,0.400763,0.368614,0.373549
2,Adaboost,0.507377,0.507377,0.520980,0.446951,0.396947,0.396947,0.323223,0.320058
3,GBM,0.595902,0.595902,0.724740,0.541538,0.398855,0.398855,0.316819,0.323199
4,SVM,0.473770,0.473770,0.514870,0.362835,0.414122,0.414122,0.295689,0.307929
5,KNN,0.497541,0.497541,0.498667,0.439992,0.406489,0.406489,0.367299,0.350522


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3043    0.9584    0.4619       361
           2     0.6129    0.0344    0.0651       553
           3     0.4423    0.0752    0.1285       306

    accuracy                         0.3180      1220
   macro avg     0.4532    0.3560    0.2185      1220
weighted avg     0.4788    0.3180    0.1984      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2925    0.8981    0.4413       157
           2     0.4286    0.0253    0.0478       237
           3     0.0714    0.0154    0.0253       130

    accuracy                         0.2844       524
   macro avg     0.2642    0.3129    0.1715       524
weighted avg     0.2992    0.2844    0.1601       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.481967,0.481967,0.633070,0.348368,0.433206,0.433206,0.279998,0.284854
1,RandomForest,0.821311,0.821311,0.868298,0.818609,0.400763,0.400763,0.368614,0.373549
2,Adaboost,0.507377,0.507377,0.520980,0.446951,0.396947,0.396947,0.323223,0.320058
3,GBM,0.595902,0.595902,0.724740,0.541538,0.398855,0.398855,0.316819,0.323199
4,SVM,0.473770,0.473770,0.514870,0.362835,0.414122,0.414122,0.295689,0.307929
5,KNN,0.497541,0.497541,0.498667,0.439992,0.406489,0.406489,0.367299,0.350522
6,GNB,0.318033,0.318033,0.478801,0.198414,0.284351,0.284351,0.299207,0.160130
